In [ ]:
import ecc
import util
from hashlib import sha256
from os import urandom

In [ ]:
## First we have a set of participants, each with his own key pair
nodes = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
#nodes = {'Z': 0, 'Y': 0, 'X': 0, 'W': 0}

for i, n in enumerate(nodes):
    nodes[n] = {'Id': i, 'Keys': ecc.PrivateKey(int.from_bytes(urandom(32), 'little'))}
    

In [ ]:
## Now let's say one of the participant, wants to make a signature
## TODO: The code only works if the signer is the first node of the set

signer = 'A'
#signer = 'Z'

## First, he needs to choose a new nonce key pair
def create_nonce(nodes, signer):
    return nodes[signer].update({'Nonce': ecc.PrivateKey(int.from_bytes(urandom(32), 'little'))})

create_nonce(nodes, signer)
print(nodes[signer])

In [ ]:
def create_index(idx, length):
    index = []
    while len(index) < length:
        index.append(idx)
        idx = (idx+1) % length
    return index
        
idx = create_index(nodes[signer]['Id'], len(nodes))
print(idx)

In [ ]:
# Let's create a few array that we will need
K = [0] * len(idx) # The K values
P = [0] * len(idx) # The pubkeys of all the nodes
e = [0] * len(idx) # The hashed message that is signed 
s = [0] * len(idx) # The signatures (remember only the signer's is a real signature, the others are garbage)

In [ ]:
# We can first collect the pubkeys of all the participant
# It worth noting that since we only need their pubkey, their active collaboration is not needed!
def sort_pubkeys(nodes, signer):
    """We first take the pubkeys in the default order and sort them so that the signer's
    is the first on the list
    """
    pubkeys = []
    temp = []
    
    for n in nodes.keys():
        temp.append(nodes[n]['Keys'].point)
    
    for i, n in enumerate(temp):
        if n == signer:
            while len(pubkeys) < len(temp):
                pubkeys.append(temp[i])
                i = (i+1) % len(temp)
            return pubkeys
        
P = sort_pubkeys(nodes, nodes[signer]['Keys'].point)

In [ ]:
# Then we need to create a message hash for the next participant. 
# This contains the message itself (here the pubkey we commit to), the public nonce K and an index that identify
# the node that is signing
def create_hash_message(K, m, n):
    """generate a hash of the nonce K of the previous node, the message m and the index of the signing node
    """
    if isinstance(K, bytes) and isinstance(m, bytes) and isinstance(n, bytes):
        to_hash = K + m + n
        return sha256(to_hash).digest()
    else:
        raise TypeError('K, m and n need to be bytes')

K[idx[0]] = nodes[signer]['Nonce'].point # The first K in our list is the "real" nonce generated by the signer
#print(f'K[{idx[0]}] is {K[idx[0]].sec().hex()}')
m = P[idx[0]].sec() # In our example the message m is simply the pubkey of the signer
#print(f'm is {m.hex()}')

# Now we compute the hash message that is getting signed by the node AFTER the signer

# K is a serialized representation (i.e. bytes) of the pubkey of the previous node
K_bin = K[idx[0]].sec()
# index is the index of the signing node, that we also need to turn into bytes
idx_bin = idx[1].to_bytes(1, 'little')

# Now we just hash the concatenation of those 3 bytes objects
e[idx[1]] = create_hash_message(K_bin, m, idx_bin) # This returns a bytes object
e[idx[1]] = int.from_bytes(e[idx[1]], 'little') # We need to interpret it as an int
print(f'e[{idx[1]}] is {e[idx[1]]}')

In [ ]:
# We need dummy signatures for all the nodes except the one that is actually signing
# We just pick a random value for all the non-signing nodes "signature"
def generate_random_sig():
    return int.from_bytes(urandom(32), 'little') % ecc.N # Don't forget to mod N!

def all_dummy_sigs(sigs):
    # The first index stay empty, it's for the real signature
    for i in range(1, len(sigs)):
        sigs[i] = generate_random_sig()
    return sigs

s = all_dummy_sigs(s)
print(s)

In [ ]:
## With this s, the hashed message e and the pubkey, we can compute a value for K
def compute_K(s, G, e, P):
    if isinstance(P, ecc.S256Point) == False:
        raise TypeError('P is not a valid S256Point')
    if isinstance(s, int) and isinstance(e, int):
        return s * G - e * P
    else:
        raise TypeError('s and e must be int')

K[idx[1]] = compute_K(s[idx[1]], ecc.G, e[idx[1]], P[idx[1]])
print(f'K[{idx[1]}] is {K[idx[1]].sec().hex()}')

In [ ]:
# we can now compute e and K for the next node
e[idx[2]] = int.from_bytes(create_hash_message(K[idx[1]].sec(), m, idx[2].to_bytes(1, 'little')), 'little')
print(f'e[{idx[2]}] is {e[idx[2]]}')

K[idx[2]] = compute_K(s[idx[2]], ecc.G, e[idx[2]], P[idx[2]])
print(f'K[{idx[2]}] is {K[idx[2]].sec().hex()}')

In [ ]:
e[idx[3]] = int.from_bytes(create_hash_message(K[idx[2]].sec(), m, idx[3].to_bytes(1, 'little')), 'little')
print(f'e[{idx[3]}] is {e[idx[3]]}')

K[idx[3]] = compute_K(s[idx[3]], ecc.G, e[idx[3]], P[idx[3]])
print(f'K[{idx[3]}] is {K[idx[3]].sec().hex()}')

In [ ]:
## Back to the signer, we calculate our own hash e that we're going to sign
e[idx[0]] = int.from_bytes(create_hash_message(K[idx[3]].sec(), m, idx[0].to_bytes(1, 'little')), 'little')
#print(f'e[{idx[0]}] is {e[idx[0]]}')

def sign(e, x, k, N):
    return ((e * x) + k) % N

# Now we make a real signature using the nonce we created in the first step
x = nodes[signer]['Keys'].secret # signer private key
k = nodes[signer]['Nonce'].secret # signer private key nonce

# The signer is the only one with access to this private material, obviously if someone had it he could 
# forge ring signatures like normal signatures

# We need K == s * G - e * P
# we can arrange the equation to look like s == e * x + k
s[0] = sign(e[idx[0]], x, k, ecc.N)
print(s)
#assert K[idx[0]] == compute_K(s[idx[0]], ecc.G, e[idx[0]], P[idx[0]])
s.append(e[idx[0]])


In [ ]:
def verify_ring(signatures, pubkeys, indexes, test=None):
    """The last item on signatures MUST be the starting e value
    The first item MUST be the signature that corresponds to the same node than the starting e value
    pubkeys list items MUST be in the same order than the signatures
    indexes list items MUST start on the node after the one for which we have e
    
    To verify the signatures we start from the given e value, compute all the e value from here until 
    we circle back to our first hash. If we got the same hash e, the signature is valid
    """
    
    sig = signatures.copy()
    pk = pubkeys.copy()
    i = indexes.copy()
    if test is not None:
        t = test.copy()
    
    target = sig.pop() # represents the given e value as int
    e = target
    #print(f'The target hash is {target}')
    
    while sig:
        #print(f'consuming signature {sig[0]}')
        #print(f'consuming pubkey {pk[0]}')
        if test is not None:
            print(f'testing index {i[0]}')
            print(f'testing against K {t[0]}')
            to_test = t.pop(0)
            try:
                assert t.pop(0) == compute_K(sig[0], ecc.G, e, pk[0])
            except AssertionError:
                raise AssertionError(f'{K} is not what it should be') 
        K = compute_K(sig.pop(0), ecc.G, e, pk.pop(0))

        if len(i) > 1:
            idx = i.pop(1).to_bytes(1, 'little')
        else:
            idx = i.pop(0).to_bytes(1, 'little')
        e = int.from_bytes(create_hash_message(K.sec(), m, idx), 'little')
    
    try:
        assert e == target
    except AssertionError:
        return False
    return True


In [ ]:
# We then publish a set containing one e value and all the signatures s values
# Anyone can verify the number of participants + 1, each value is of 32B

print(verify_ring(s, P, idx))

In [ ]:
print(ring2)

In [ ]:
ring2 = (s, P, idx)

In [ ]:
# Save our first ring signatures and generate another one

ring1 = (s, P, idx)

In [ ]:
## Now let's create another set of participant with their own keys

    
def create_nonce(nodes, node):
    return nodes[node].update({'Nonce': ecc.PrivateKey(int.from_bytes(urandom(32), 'little'))})

def all_dummy_sigs(nb):
    sigs = []
    while len(sigs) < nb:
        sigs.append(generate_random_sig())
    
    return sigs

def create_index(nodes):
    temp = list(nodes.keys())
    index = []
    for i, n in enumerate(temp):
        if 'Nonce' in nodes[n]:
            while len(index) < len(temp):
                i = (i+1) % len(temp)
                index.append(temp[i].encode('utf-8'))
            return index
        
def sorted_pubkeys(nodes, signer):
    pubkeys = []
    temp = []
    
    for n in nodes.keys():
        temp.append(nodes[n]['Keys'].point)
    
    for i, n in enumerate(temp):
        if n == signer:
            print(f"First pubkey {temp[i]}")
            while len(pubkeys) < len(temp):
                pubkeys.append(temp[i])
                i = (i+1) % len(temp)
            return pubkeys

def create_ring_sig(nodes, signer):
    print(f"Signer is {signer}, pubkey is {nodes[signer]['Keys'].point}")
    # create the message (the serialized pubkey of the signer)
    m = nodes[signer]['Keys'].point.sec()
    #print(m)
    # create a nonce for the signer only
    create_nonce(nodes, signer)
    K_sig = nodes[signer]['Nonce'].point
    # Get the pubkeys of all the participant
    pubkeys = sorted_pubkeys(nodes, nodes[signer]['Keys'].point)
    # Create a list of index in the right order
    index = create_index(nodes)
    i = index.copy()
    print(index)
    
    # Create a hash of the K, m and an index
    e = int.from_bytes(create_hash_message(K_sig.sec(), m, i.pop(0)), 'little')
    #print(e)
    
    # We generate random signatures for the other participants
    signatures = all_dummy_sigs(len(index)-1)
    
    # Now we can compute K and e values for each node
    sigs = signatures.copy()
    pk = pubkeys.copy()
    test = []
    while sigs:
        #print(f'consuming signature {sigs[0]}')
        #print(f'consuming pubkey {pk[0]}')
        print(f"Is pk[1] == nodes['Y']['Keys'].point True ?", pk[1] == nodes['Y']['Keys'].point)
        K = compute_K(sigs.pop(0), ecc.G, e, pk.pop(1))
        #print(f'result K {K}')
        test.append(K)
        #print(i[0])
        e = int.from_bytes(create_hash_message(K.sec(), m, i.pop(0)), 'little')
    
    #print(test)
    x = nodes[signer]['Keys'].secret # private key
    k = nodes[signer]['Nonce'].secret # private key nonce

    s = (e * x + k) % ecc.N
    assert K_sig == compute_K(s, ecc.G, e, nodes[signer]['Keys'].point)
    test.insert(0, K_sig)
    signatures.insert(0, s)
    signatures.append(e)
    #for n, i in enumerate(index):
    #    print(f'Index {n} is {i}')
    #for n, i in enumerate(pubkeys):
    #    print(f'Pubkey {n} is {i}')
    #for n, i in enumerate(signatures):
    #    print(f'Sig {n} is {i}')
    for n, i in enumerate(test):
        print(f'K {n} is {i}')
    return signatures, pubkeys, index, test


In [ ]:
participants2 = {'Z': 0, 'Y': 0, 'X': 0, 'W': 0}

for p in participants2:
    participants2[p] = {'Keys': ecc.PrivateKey(int.from_bytes(urandom(32), 'little'))}
    
signatures, pubkeys, index, test = create_ring_sig(participants2, 'Z')

print(verify_ring(signatures, pubkeys, index, test))